## Preparation

In [1]:
import numpy as np
import pandas as pd
from tqdm.contrib.concurrent import thread_map
from rdkit import Chem
from project_filtering import BASE_DIR
import numpy as np
import pandas as pd
from rdkit.Chem import MolToSmiles, MolFromSmiles
from rdkit.Chem import AllChem
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import thread_map
import mordred as md

from project_filtering import BASE_DIR
from mordred import Autocorrelation
from mordred import BaryszMatrix

In [ ]:
properties_df = pd.read_csv(BASE_DIR / "data/input/gdb9.sdf.csv")
properties_df

In [3]:
supplier = Chem.SDMolSupplier(BASE_DIR / "data/input/gdb9.sdf")


In [4]:
molecules = np.array([x for x in tqdm(supplier)])
print(np.count_nonzero(molecules))

  0%|          | 0/133885 [00:00<?, ?it/s]

[14:44:22] Explicit valence for atom # 1 C, 5, is greater than permitted
[14:44:22] ERROR: Could not sanitize molecule ending on line 9097
[14:44:22] ERROR: Explicit valence for atom # 1 C, 5, is greater than permitted
[14:44:22] Explicit valence for atom # 1 C, 5, is greater than permitted
[14:44:22] ERROR: Could not sanitize molecule ending on line 35785
[14:44:22] ERROR: Explicit valence for atom # 1 C, 5, is greater than permitted
[14:44:22] Explicit valence for atom # 4 C, 5, is greater than permitted
[14:44:22] ERROR: Could not sanitize molecule ending on line 62866
[14:44:22] ERROR: Explicit valence for atom # 4 C, 5, is greater than permitted
[14:44:22] Explicit valence for atom # 2 C, 5, is greater than permitted
[14:44:22] ERROR: Could not sanitize molecule ending on line 66832
[14:44:22] ERROR: Explicit valence for atom # 2 C, 5, is greater than permitted
[14:44:22] Explicit valence for atom # 2 C, 5, is greater than permitted
[14:44:22] ERROR: Could not sanitize molecule en

132737


[14:44:23] Explicit valence for atom # 2 C, 5, is greater than permitted
[14:44:23] ERROR: Could not sanitize molecule ending on line 664156
[14:44:23] ERROR: Explicit valence for atom # 2 C, 5, is greater than permitted
[14:44:23] Explicit valence for atom # 2 C, 5, is greater than permitted
[14:44:23] ERROR: Could not sanitize molecule ending on line 664543
[14:44:23] ERROR: Explicit valence for atom # 2 C, 5, is greater than permitted
[14:44:23] Explicit valence for atom # 2 C, 5, is greater than permitted
[14:44:23] ERROR: Could not sanitize molecule ending on line 664576
[14:44:23] ERROR: Explicit valence for atom # 2 C, 5, is greater than permitted
[14:44:23] Explicit valence for atom # 2 C, 5, is greater than permitted
[14:44:23] ERROR: Could not sanitize molecule ending on line 664869
[14:44:23] ERROR: Explicit valence for atom # 2 C, 5, is greater than permitted
[14:44:23] Explicit valence for atom # 2 C, 5, is greater than permitted
[14:44:23] ERROR: Could not sanitize molecu

In [5]:
valid_mask = np.array([i for i, x in enumerate(molecules) if x is not None])
valid_molecules = molecules[valid_mask]
valid_noniso_smiles = np.array(
    list(
        map(
            lambda x: Chem.MolToSmiles(x, isomericSmiles=False, canonical=True),
            tqdm(valid_molecules),
        )
    )
)


  0%|          | 0/132737 [00:00<?, ?it/s]

In [6]:
unique_noniso_smiles, unique_idx = np.unique(valid_noniso_smiles, return_index=True)
len(unique_noniso_smiles)

132398

In [7]:
clean_mask = np.sort(valid_mask[unique_idx])
len(clean_mask)

132398

In [8]:
values = -1 * properties_df.loc[:, "g298_atom"] / 1000
values.iloc[clean_mask].describe()

count    132398.000000
mean          1.630663
std           0.220153
min           0.201407
25%           1.489943
50%           1.633793
75%           1.772368
max           2.417122
Name: g298_atom, dtype: float64

In [9]:
molecules = molecules[clean_mask]
values = values.to_numpy()[clean_mask]

In [10]:
clean_mask

array([     0,      1,      2, ..., 133882, 133883, 133884])

In [11]:
properties_df.iloc[clean_mask,:]

,mol_id,A,B,C,mu,alpha,homo,lumo,gap,r2,zpve,u0,u298,h298,g298,cv,u0_atom,u298_atom,h298_atom,g298_atom
0,gdb_1,157.71180,157.709970,157.706990,0.0000,13.21,-0.3877,0.1171,0.5048,35.3641,0.044749,-40.478930,-40.476062,-40.475117,-40.498597,6.469,-395.999595,-398.643290,-401.014647,-372.471772
1,gdb_2,293.60975,293.541110,191.393970,1.6256,9.46,-0.2570,0.0829,0.3399,26.1563,0.034358,-56.525887,-56.523026,-56.522082,-56.544961,6.316,-276.861363,-278.620271,-280.399259,-259.338802
2,gdb_3,799.58812,437.903860,282.945450,1.8511,6.31,-0.2928,0.0687,0.3615,19.0002,0.021375,-76.404702,-76.401867,-76.400922,-76.422349,6.002,-213.087624,-213.974294,-215.159658,-201.407171
3,gdb_4,0.00000,35.610036,35.610036,0.0000,16.28,-0.2845,0.0506,0.3351,59.5248,0.026841,-77.308427,-77.305527,-77.304583,-77.327429,8.574,-385.501997,-387.237686,-389.016047,-365.800724
4,gdb_5,0.00000,44.593883,44.593883,2.8937,12.99,-0.3604,0.0191,0.3796,48.7476,0.016601,-93.411888,-93.409370,-93.408425,-93.431246,6.278,-301.820534,-302.906752,-304.091489,-288.720028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133878,gdb_133879,3.52243,2.185850,1.902520,1.4529,77.63,-0.2229,0.0977,0.3206,796.9713,0.151939,-364.733608,-364.728220,-364.727276,-364.762857,24.820,-1802.904893,-1815.522844,-1825.598756,-1678.637403
133879,gdb_133880,3.56068,2.195510,1.920670,1.3774,73.24,-0.2427,0.0904,0.3331,777.3913,0.139809,-384.609506,-384.604306,-384.603362,-384.638652,23.968,-1737.300710,-1749.148080,-1758.630996,-1619.706151
133882,gdb_133883,3.67118,2.143140,1.895010,1.2480,73.60,-0.2233,0.0720,0.2953,780.3553,0.140458,-380.753918,-380.748619,-380.747675,-380.783148,23.972,-1667.045429,-1678.830048,-1688.312964,-1549.143391
133883,gdb_133884,3.52845,2.151310,1.865820,1.9576,77.40,-0.2122,0.0881,0.3003,803.1904,0.152222,-364.720374,-364.714974,-364.714030,-364.749650,24.796,-1794.600439,-1807.210860,-1817.286772,-1670.349892


## Clean Dataframe

In [11]:
smiles = np.array(
    list(
        map(
            lambda x: Chem.MolToSmiles(x, isomericSmiles=True, canonical=True), tqdm(molecules)
        )
    )
)

  0%|          | 0/132398 [00:00<?, ?it/s]

In [ ]:
properties_df = pd.read_csv(BASE_DIR / "data/input/gdb9.sdf.csv")
properties_df

In [ ]:
clean_df = properties_df.iloc[clean_mask, :].copy()
clean_df.insert(1, "smiles", smiles)
clean_df = clean_df.sort_values("smiles")
clean_df.to_csv(BASE_DIR / "data/processed/qm9-clean.csv", sep=";", index=False)
clean_df

In [ ]:
clean_unsorted_df = properties_df.iloc[clean_mask, :].copy()
clean_unsorted_df.insert(1, "smiles", smiles)
# clean_unsorted_df = clean_unsorted_df.sort_values("smiles")
clean_unsorted_df.to_csv(BASE_DIR / "data/processed/qm9-clean-unsorted.csv", sep=";")
clean_unsorted_df

In [29]:
np.save(
    BASE_DIR/"data/processed/qm9-mask.npy",
    clean_mask
)

## Shuffled Dataset

In [ ]:
properties_df = pd.read_csv(BASE_DIR / "data/input/gdb9.sdf.csv")
properties_df

In [32]:
clean_properties_df = properties_df.iloc[clean_mask,:].copy(deep=True)
rng = np.random.default_rng(seed=42)
for i in [0.5, 1, 2.5, 5, 10]:
    new_properties_df = clean_properties_df.copy(deep=True)
    shuffle_idx = rng.choice(
        len(clean_properties_df),
        size=int((i / 100) * len(clean_properties_df)),
        replace=False,
        shuffle=True,
    )
    new_properties_df.iloc[sorted(shuffle_idx), 1:] = clean_properties_df.iloc[
        shuffle_idx, 1:
    ]
    new_properties_df.to_csv(BASE_DIR / f"data/processed/qm9-shuffle-{i}.csv", sep=";")

In [34]:
for i in [0.5, 1,2.5,5,10]:
    new_synth_df = pd.read_csv(BASE_DIR/f"data/processed/qm9-shuffle-{i}.csv", index_col=0, sep = ";")
    print(np.count_nonzero(1-np.isclose(new_synth_df["g298_atom"],clean_properties_df["g298_atom"])))

661
1322
3308
6619
13238


## Variable Constant Value Addition

In [ ]:
properties_df = pd.read_csv(BASE_DIR / "data/input/gdb9.sdf.csv")
properties_df

In [13]:
properties_df["g298_atom"].describe()

count    133885.000000
mean      -1629.388196
std         220.207088
min       -2417.121997
25%       -1771.350603
50%       -1632.224955
75%       -1488.291333
max        -201.407171
Name: g298_atom, dtype: float64

In [14]:
spread = properties_df["g298_atom"].max()-properties_df["g298_atom"].min()
spread, spread/30

(2215.7148262210003, 73.85716087403334)

In [17]:
clean_properties_df = properties_df.iloc[clean_mask, :].copy(deep=True)
rng = np.random.default_rng(seed=42)
for i in [0.5, 1, 2.5, 5, 10]:
    for delta in [0.005, 0.0075, 0.01, 0.025, 0.05, 0.1, 0.25]:
        new_mix_df = clean_properties_df.copy(deep=True)
        mod_idx = rng.choice(
            len(clean_properties_df),
            size=int((i / 100) * len(clean_properties_df)),
            replace=False,
            shuffle=False,
        )
        new_mix_df.iloc[mod_idx, 1:] = new_mix_df.iloc[mod_idx, 1:] - 1000*delta
        new_mix_df.to_csv(
            BASE_DIR / f"data/processed/qm9-variable-mu-{delta}-{i}.csv", sep=";"
        )

In [15]:
clean_properties_df = properties_df.iloc[clean_mask, :].copy(deep=True)
rng = np.random.default_rng(seed=42)
for i in [0.5, 1, 2.5, 5, 10]:
    for delta in [0.0125, 0.015, 0.075, 0.08]:
        new_mix_df = clean_properties_df.copy(deep=True)
        mod_idx = rng.choice(
            len(clean_properties_df),
            size=int((i / 100) * len(clean_properties_df)),
            replace=False,
            shuffle=False,
        )
        new_mix_df.iloc[mod_idx, 1:] = new_mix_df.iloc[mod_idx, 1:] - 1000 * delta
        new_mix_df.to_csv(
            BASE_DIR / f"data/processed/qm9-variable-mu-{delta}-{i}.csv", sep=";"
        )

In [14]:
clean_properties_df = properties_df.iloc[clean_mask, :].copy(deep=True)
rng = np.random.default_rng(seed=42)
for i in [0.5, 1, 2.5, 5, 10]:
    for delta in [0.125, 0.15, 0.175, 0.2]:
        new_mix_df = clean_properties_df.copy(deep=True)
        mod_idx = rng.choice(
            len(clean_properties_df),
            size=int((i / 100) * len(clean_properties_df)),
            replace=False,
            shuffle=False,
        )
        new_mix_df.iloc[mod_idx, 1:] = new_mix_df.iloc[mod_idx, 1:] - 1000 * delta
        new_mix_df.to_csv(
            BASE_DIR / f"data/processed/qm9-variable-mu-{delta}-{i}.csv", sep=";"
        )

In [15]:
clean_properties_df = properties_df.iloc[clean_mask, :].copy(deep=True)
for i in [0.5, 1, 2.5, 5, 10]:
    for delta in [0.0175, 0.0200, 0.0225]:
        rng = np.random.default_rng(seed=42)
        new_mix_df = clean_properties_df.copy(deep=True)
        mod_idx = rng.choice(
            len(clean_properties_df),
            size=int((i / 100) * len(clean_properties_df)),
            replace=False,
            shuffle=False,
        )
        new_mix_df.iloc[mod_idx, 1:] = new_mix_df.iloc[mod_idx, 1:] - 1000 * delta
        new_mix_df.to_csv(
            BASE_DIR / f"data/processed/qm9-variable-mu-{delta}-{i}.csv", sep=";"
        )